In [5]:
import os, inspect, sys
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [6]:
%matplotlib inline

In [12]:
from keras.engine.saving import load_model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from src.lime_timeseries_optimized import LimeTimeSeriesExplanation
import math
import matplotlib
import numpy as np
import scipy
from plotly import graph_objs as go
from plotly.offline import plot, iplot
import plotly.io as pio
from scipy import signal
import plotly.offline as offline
from src.subsampling_normalization import get_chunks, downsample_chunks, chunks_magnitude_normalization
from IPython.display import HTML
from tensorflow.python.saved_model import tag_constants


In [38]:
def plot_lime_explanation_mpl(error_idx):
    instance= test_set[error_idx]
    explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
    explanations = explainer.explain_instance(timeseries=instance, num_features=10, training_set=TRAINING_SET,
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_predict)
    explanations = explanations.as_list()
    plt.figure(figsize=(20, 7))
    values_per_slice = math.ceil(len(instance) / 40)
    plt.plot(instance, color='b', label='Explained instance')
    plt.legend(loc='lower left')
    for i in range(10):
        feature, weight = explanations[i]
        start = feature * values_per_slice
        weight = abs(weight)
        weight *= 10**25
        if weight < 0.1:
            weight = 0.1
        end = start + values_per_slice
        plt.title("Prediction: {} Label: {}, Index: {}".format(predictions[error_idx], test_labels[error_idx], error_idx))
        plt.axvspan(start , end, color='red', alpha=weight)
    fig =  plt.gcf()
    plt.close()
    return fig

In [8]:
offline.init_notebook_mode(connected=True)

In [9]:
matplotlib.rcParams.update({'font.size': 22})

In [39]:
def get_lime_explanation(instance, num_features=10, num_slices=40, num_samples=250):
    explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
    explanations = explainer.explain_instance(timeseries=instance, num_features=num_features, training_set=TRAINING_SET,
                                              num_samples=num_samples, num_slices=num_slices,
                                              classifier_fn=lime_predict)
    return plot_lime_explanation(explanations, instance)


def lime_predict(instances):
    labels = []
    for instance in instances:
        instance = np.reshape(instance, newshape=(1, instance.shape[0], 1))
        with GRAPH.as_default():
            label = MODEL.predict(instance)
        labels.append(label)
    return np.array(labels).reshape(len(instances), 2)

def plot_lime_explanation(explanations, instance, prediction, label, error_idx, num_slices=40):
    exp = explanations.as_list(label=0)

    trace = go.Scatter(
        x=np.arange(0, 10000, 1),
        y=np.squeeze(instance),
        mode="lines"
    )
    data = [trace]
    if prediction == 1 and label ==0:
        mistake = 'FN'
    elif prediction == 0 and label == 1:
        mistake = 'FP'
    layout = {'font':{'size': 20},'title': 'Predykcja: {}, Etykieta: {}, Rodzaj błędu: {}'.format(prediction, label, mistake),
              'xaxis': {'title': 'Próbka', 'showgrid': False},
              'yaxis': {'title': 'Natężenie dźwięku', 'showgrid': False}, 'shapes': []}
    shape = {'type': 'rect', 'xref': 'x', 'yref': 'paper', 'x0': 0, 'y0': 0, 'x1': 0, 'y1': 1, 'fillcolor': '#f24d50',
             'opacity': 0.0, 'line': {'width': 0}, 'layer': 'below'}
    values_per_slice = math.ceil(len(instance) / num_slices)
    weights = [abs(sample[1]) * 10 ** 25 for sample in exp]
    normalized_weights = [(weight - min(weights)) / (max(weights) - min(weights)) for weight in weights]
    for i in range(len(exp)):
        feature, _ = exp[i]
        weight = normalized_weights[i]
        if weight < 0.1:
            weight = 0.1
        start = feature * values_per_slice
        end = start + values_per_slice
        shape1 = shape.copy()
        shape1.update({'x0': start, 'x1': end, 'opacity': weight})
        layout['shapes'].append(shape1)
    fig = go.Figure(data=data, layout=layout)
#     pio.write_image(fig, 'images/error_{}.png'.format(error_idx))
    return iplot(fig)

In [19]:
MODEL = load_model('/home/jczestochowska/workspace/heartbeat_classification/src/flask-app/convo_weights.h5')

In [10]:
test_set = np.load('/home/jczestochowska/workspace/heartbeat_classification/data/processed/preprocessed/physionet/serialized/no_feature_extraction/test.npy')
test_labels = np.load('/home/jczestochowska/workspace/heartbeat_classification/data/processed/preprocessed/physionet/serialized/no_feature_extraction/test_labels.npy')

In [21]:
test_set.shape

(4332, 10000)

In [31]:
TRAINING_SET = np.load('/home/jczestochowska/workspace/heartbeat_classification/data/processed/preprocessed/physionet/serialized/no_feature_extraction/train.npy')

In [22]:
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1], 1)
predictions_probs = MODEL.predict(test_set)

In [23]:
predictions = np.argmax(predictions_probs, axis=1)

In [24]:
errored_examples = np.where(predictions != test_labels)

In [25]:
errored_examples = errored_examples[0]

In [121]:
fp =0
fn = 0
for errored_example in errored_examples:
    label = test_labels[errored_example]
    prediction = np.argmax(MODEL.predict(test_set[errored_example].reshape(1,10000,1)))
    if prediction == 1 and label == 0:
        mistake = 'FN'
        fn += 1
    elif prediction == 0 and label == 1:
        mistake = 'FP'
        fp += 1

In [122]:
print(fn)
print(fp)

110
225


In [111]:
type(errored_examples)

numpy.ndarray

In [26]:
global GRAPH
GRAPH = tf.get_default_graph()

In [ ]:
explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
explanations = explainer.explain_instance(timeseries=instance, num_features=10, training_set=TRAINING_SET,
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_predict)

In [ ]:
explanations_list = explanations.as_list()

In [ ]:
for error_idx in errored_examples:
    plt.figure(figsize=(20,7))
    plt.plot(test_set[error_idx])
    plt.title("Prediction: {} Label: {}, Index: {}".format(predictions[error_idx], test_labels[error_idx], error_idx))
    plt.show()

In [ ]:
with open("images/error_plots.html") as f:
    for error_idx in errored_examples:
        html = plot_lime_explanation(explanations, error_idx=error_idx, prediction=predictions[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])
        f.write(html)
        f.write('\n')

In [113]:
import random
indices = random.sample(list(errored_examples), 100)
for error_idx in indices:
    print(error_idx)
    explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
    explanations = explainer.explain_instance(timeseries=test_set[error_idx], num_features=10, training_set=TRAINING_SET[:100],
                                              num_samples=250, num_slices=40,
                                              classifier_fn=lime_predict)
    plot_lime_explanation(explanations=explanations, error_idx=error_idx, prediction=predictions[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])

In [126]:
error_idx = 3476
explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
explanations = explainer.explain_instance(timeseries=test_set[error_idx], num_features=10, training_set=TRAINING_SET[:100],
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_predict)
plot_lime_explanation(explanations=explanations, error_idx=error_idx, prediction=predictions[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])
print(explanations.as_list())

In [152]:
error_idx = 2318
explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
explanations = explainer.explain_instance(timeseries=test_set[error_idx], num_features=10, training_set=TRAINING_SET[:100],
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_predict)
plot_lime_explanation(explanations=explanations, error_idx=error_idx, prediction=predictions[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])
print(explanations.as_list())

In [145]:
error_idx = 1001
explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
explanations = explainer.explain_instance(timeseries=test_set[error_idx], num_features=10, training_set=TRAINING_SET[:100],
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_predict)
plot_lime_explanation(explanations=explanations, error_idx=error_idx, prediction=predictions[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])
print(explanations.as_list())

In [146]:
error_idx = 1001
explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
explanations = explainer.explain_instance(timeseries=test_set[error_idx], num_features=10, training_set=TRAINING_SET[:100],
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_predict)
plot_lime_explanation(explanations=explanations, error_idx=error_idx, prediction=predictions[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])
print(explanations.as_list())

In [33]:
from librosa import feature

def lime_lr_predict(instances):
    predictions = []
    graph = tf.Graph()
    with graph.as_default():
        with tf.Session(graph=graph) as sess:
            tf.saved_model.loader.load(
                sess,
                [tag_constants.SERVING],
                "/home/jczestochowska/workspace/heartbeat_classification/models/logistic_regression_weights"
            )
            i=0
            for instance in instances:
                instance = feature.mfcc(instance, 2000)
                w = graph.get_tensor_by_name("Variable:0")
                b = graph.get_tensor_by_name("Variable_1:0")
                instance = instance.reshape(1, 400)
                logits = tf.matmul(instance, w) + b
                prediction = sess.run(tf.sigmoid(logits))
                predictions.append(prediction)
                print(i)
                i +=1
    return np.array(predictions).reshape(len(instances), 2)

In [34]:
predictions_lr = lime_lr_predict(test_set)

In [35]:
errors_lr = np.where(predictions_lr != test_labels)

In [ ]:
len(errors_lr[0])

In [41]:
error_idx = 1
explainer = LimeTimeSeriesExplanation(feature_selection='auto', verbose=False)
explanations = explainer.explain_instance(timeseries=test_set[error_idx], num_features=10, training_set=TRAINING_SET[:100],
                                          num_samples=250, num_slices=40,
                                          classifier_fn=lime_lr_predict)
plot_lime_explanation(explanations=explanations, error_idx=error_idx, prediction=predictions_lr[error_idx], label=test_labels[error_idx],instance=test_set[error_idx])

In [42]:
explanations.as_list()

In [ ]:
import matplotlib.backends.backend_pdf
import random
random_erored_examples = random.sample(list(errored_examples), k=30)

In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages("errors_lime.pdf")
for error_idx in random_erored_examples: ## will open an empty extra figure :(
    pdf.savefig(plot_lime_explanation_mpl(error_idx))
pdf.close()